In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
import datetime
from sklearn.preprocessing import LabelEncoder
from pyecharts import options as opts
from pyecharts.charts import Graph
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot


In [2]:
#############Loading data from Front and Back End#############
dfFront = pd.read_csv("data/WebFrontend_for_DFADR.csv")
dfBack = pd.read_csv("data/WebBackend_for_DFADR.csv")

dfFront.columns = ['tih', 'sip', 'dip', 'num','url','label']
dfFront_DF = dfFront[['tih', 'sip', 'dip', 'num']]
dfBack_DF = dfBack[['tih', 'sip', 'dip', 'num']]

dfFront_DF['tih'] = dfFront_DF['tih'].astype('datetime64[ns]')
dfBack_DF['tih'] = dfBack_DF['tih'].astype('datetime64[ns]')

### Get the sum of num for each tih-sip-dip pair
FrontGroups_sip_dip_sum = pd.DataFrame(dfFront_DF.groupby(['tih','sip','dip'])['num'].sum())
FrontGroups_sip_dip_sum.columns = [''.join(col) for col in FrontGroups_sip_dip_sum.columns]
FrontGroups_sip_dip_sum = FrontGroups_sip_dip_sum.reset_index()

BackGroups_sip_dip_sum = pd.DataFrame(dfBack_DF.groupby(['tih','sip','dip'])['num'].sum())
BackGroups_sip_dip_sum.columns = [''.join(col) for col in BackGroups_sip_dip_sum.columns]
BackGroups_sip_dip_sum = BackGroups_sip_dip_sum.reset_index()

dfFront_DF = FrontGroups_sip_dip_sum
dfBack_DF = BackGroups_sip_dip_sum

dfFront_DF.shape[0]
# dfBack_DF.shape[0]

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


608

### Statistics

In [3]:
#######Merge the list of all IPs ##### i.e. the total number of IPs at this level
dfFront_DF_IP = pd.concat([dfFront_DF['sip'],dfFront_DF['dip']],axis=0).drop_duplicates()
dfFront_DF_IP.reset_index(drop=True, inplace=True)

dfBack_DF_IP = pd.concat([dfBack_DF['sip'],dfBack_DF['dip']],axis=0).drop_duplicates()
dfBack_DF_IP.reset_index(drop=True, inplace=True)

dfAll_DF_IP = pd.merge(dfFront_DF_IP.to_frame(),dfBack_DF_IP.to_frame(),how='outer')   
dfAll_DF_IP.shape[0]

51

In [4]:
dfBack_time=dfBack_DF[['tih']]
dfBack_time=dfBack_time.drop_duplicates()
dfBack_time.sort_values(by=['tih'],inplace=True,ascending=True)
dfBack_time.reset_index(drop=True, inplace=True)
dfBack_time.head()

,tih
0,2019-05-30 23:00:00
1,2019-05-31 00:00:00
2,2019-05-31 11:00:00
3,2019-05-31 14:00:00
4,2019-06-03 17:00:00


### DFADR&AttackGraph

In [5]:
#############Defining Parameters and Functions#############
dfabcorrect=['10.49.253.233','10.49.253.35','10.49.223.194','10.49.212.162','10.49.21.15',
            '10.49.141.206','123.58.173.111']
dfabcorrect_set = set(dfabcorrect)
#######Normalization Function
min_max_scaler_13 = preprocessing.MinMaxScaler(feature_range=(1,3), copy=True)
min_max_scaler_2030 = preprocessing.MinMaxScaler(feature_range=(20,30), copy=True)
# min_max_scaler_028 = preprocessing.MinMaxScaler(feature_range=(0.2,0.8), copy=True)

IPnumi = dfAll_DF_IP.shape[0]

#######Metric Function
def Metric(IPnum,abIP,TP,FN):
    FP=abIP-TP
    TN=IPnum-abIP-FN#######是IPnum有问题
    R= TP/(TP+FN)
    TPR = TP/(TP+FN)
    FPR = FP/(TN+FP)
    P= TP/(TP+FP)
    return round(R, 4),round(TPR, 4),round(FPR, 4),round(P, 4)

In [8]:
#######Define logdf to store logs for visualization
Logdf = pd.DataFrame(columns=('t','ROC_AUC','PRC_AUC','AbIPFind_time','Visual_time'))###TimeCost does not include the ROC PRC process

for t in range(0,51):######Test for 0-6 ###### Of course you can increase the number of scanners to be excluded, but the results will not be better
    
    time_start1 = time.time()
    
    df_Keep_FinalAll_0 = pd.DataFrame(columns=('tih_x','sip_x','dip_x','num_x','tih_y','sip_y','dip_y','num_y'))    
    df_Keep_FinalAll_1 = pd.DataFrame(columns=('tih_x','sip','dip_x','num_x','tih_y','dip_y','num_y'))
    df_Keep_FinalAll_2 = pd.DataFrame(columns=('tih_x','sip_x','dip_x','num_x','tih_y','sip_y','dip_y','num_y'))
    df_Keep_FinalAll_3 = pd.DataFrame(columns=('tih_x','sip_x','dip','num_x','tih_y','sip_y','num_y'))
    for timei in dfBack_time['tih']:### Based on Backend's time
        #####Retain front-end and back-end records within the time frame
        dfFront_Keepi = dfFront_DF[(dfFront_DF['tih']<=(timei + datetime.timedelta(hours=t)).strftime("%Y-%m-%d %H:%M:%S"))&
                                (dfFront_DF['tih']>=(timei - datetime.timedelta(hours=t)).strftime("%Y-%m-%d %H:%M:%S"))
                             ]
        dfBack_DFi = dfBack_DF[dfBack_DF['tih'] == timei]
        Merge0 = pd.merge(dfFront_Keepi,dfBack_DFi,left_on='sip',right_on='dip',how='inner')
        Merge1 = pd.merge(dfFront_Keepi,dfBack_DFi,left_on='sip',right_on='sip',how='inner')
        Merge2 = pd.merge(dfFront_Keepi,dfBack_DFi,left_on='dip',right_on='sip',how='inner') 
        Merge3 = pd.merge(dfFront_Keepi,dfBack_DFi,left_on='dip',right_on='dip',how='inner') 

        if Merge0.shape[0] != 0:
            df_Keep_FinalAll_0 = pd.concat([df_Keep_FinalAll_0, Merge0], axis=0)
        if Merge1.shape[0] != 0:
            df_Keep_FinalAll_1 = pd.concat([df_Keep_FinalAll_1, Merge1], axis=0)
        if Merge2.shape[0] != 0:
            df_Keep_FinalAll_2 = pd.concat([df_Keep_FinalAll_2, Merge2], axis=0)
        if Merge3.shape[0] != 0:
            df_Keep_FinalAll_3 = pd.concat([df_Keep_FinalAll_3, Merge3], axis=0)


    df_Keep_FinalAll_1['sip_y'] = df_Keep_FinalAll_1['sip']
    df_Keep_FinalAll_1.rename(columns={'sip':'sip_x'}, inplace = True)

    df_Keep_FinalAll_3['dip_y'] = df_Keep_FinalAll_3['dip']
    df_Keep_FinalAll_3.rename(columns={'dip':'dip_x'}, inplace = True)

#######Integrate, Rename
    df_Keep_FinalAll = pd.concat([df_Keep_FinalAll_1,df_Keep_FinalAll_2],axis=0)
    df_Keep_FinalAll = pd.concat([df_Keep_FinalAll,df_Keep_FinalAll_3],axis=0)
    df_Keep_FinalAll = pd.concat([df_Keep_FinalAll,df_Keep_FinalAll_0],axis=0)
    df_Keep_FinalAll.reset_index(drop=True, inplace=True)
    df_Keep_FinalAll = df_Keep_FinalAll.reindex(columns = ['tih_x','sip_x','dip_x','num_x','tih_y','sip_y','dip_y','num_y'])
#######Divide the front-end subset
    df_Keep_FinalAll_Front = df_Keep_FinalAll[['tih_x','sip_x','dip_x','num_x']].drop_duplicates()
    #####Summing the num of the same tih-sip-dip
    FrontGroups_tihsipdip_sum = pd.DataFrame(df_Keep_FinalAll_Front.groupby(['tih_x','sip_x','dip_x'])['num_x'].sum())
    FrontGroups_tihsipdip_sum.columns = [''.join(col) for col in FrontGroups_tihsipdip_sum.columns]
    FrontGroups_tihsipdip_sum = FrontGroups_tihsipdip_sum.reset_index()
    df_Keep_FinalAll_Front = FrontGroups_tihsipdip_sum
    #####Normalization
    df_Keep_FinalAll_Front[['num_x']]=min_max_scaler_13.fit_transform(df_Keep_FinalAll_Front[['num_x']])
    df_Keep_FinalAll_Front['tih_x'] = df_Keep_FinalAll_Front['tih_x'].astype('datetime64[ns]').apply(lambda x: datetime.datetime.strftime(x,'%dd%H'))

#######Divide the back-end subset
    df_Keep_FinalAll_Back = df_Keep_FinalAll[['tih_y','sip_y','dip_y','num_y']].drop_duplicates()
    #####Summing the num of the same tih-sip-dip
    BackGroups_tihsipdip_sum = pd.DataFrame(df_Keep_FinalAll_Back.groupby(['tih_y','sip_y','dip_y'])['num_y'].sum())
    BackGroups_tihsipdip_sum.columns = [''.join(col) for col in BackGroups_tihsipdip_sum.columns]
    BackGroups_tihsipdip_sum = BackGroups_tihsipdip_sum.reset_index()
    df_Keep_FinalAll_Back = BackGroups_tihsipdip_sum
    df_Keep_FinalAll_Back[['num_y']]=min_max_scaler_13.fit_transform(df_Keep_FinalAll_Back[['num_y']])
    df_Keep_FinalAll_Back['tih_y'] = df_Keep_FinalAll_Back['tih_y'].astype('datetime64[ns]').apply(lambda x: datetime.datetime.strftime(x,'%dd%H'))
    
#############Find the list of IPs and the numSum of each IP
    FrontGroups_sip_sum = pd.DataFrame(df_Keep_FinalAll_Front.groupby(['sip_x'])['num_x'].sum())
    FrontGroups_dip_sum= pd.DataFrame(df_Keep_FinalAll_Front.groupby(['dip_x'])['num_x'].sum())
    FrontGroups_sip_sum.columns = [''.join(col) for col in FrontGroups_sip_sum.columns]
    FrontGroups_dip_sum.columns = [''.join(col) for col in FrontGroups_dip_sum.columns]
    FrontGroups_sip_sum = FrontGroups_sip_sum.reset_index()
    FrontGroups_dip_sum = FrontGroups_dip_sum.reset_index()

    FrontGroups_sip_sum.columns =['ip','numSum']
    FrontGroups_dip_sum.columns =['ip','numSum']

    FrontGroups_IP_sum = pd.concat([FrontGroups_sip_sum, FrontGroups_dip_sum], axis=0)
    FrontGroups_IP_sum.reset_index(drop=True, inplace=True)


    BackGroups_sip_sum= pd.DataFrame(df_Keep_FinalAll_Back.groupby(['sip_y'])['num_y'].sum()) 
    BackGroups_dip_sum= pd.DataFrame(df_Keep_FinalAll_Back.groupby(['dip_y'])['num_y'].sum()) 
    BackGroups_sip_sum.columns = [''.join(col) for col in BackGroups_sip_sum.columns]
    BackGroups_dip_sum.columns = [''.join(col) for col in BackGroups_dip_sum.columns]
    BackGroups_sip_sum = BackGroups_sip_sum.reset_index()
    BackGroups_dip_sum = BackGroups_dip_sum.reset_index()
    BackGroups_sip_sum.columns =['ip','numSum']
    BackGroups_dip_sum.columns =['ip','numSum']
    BackGroups_IP_sum = pd.concat([BackGroups_sip_sum, BackGroups_dip_sum], axis=0)
    BackGroups_IP_sum.reset_index(drop=True, inplace=True)

    Groups_IP_sum = pd.merge(FrontGroups_IP_sum,BackGroups_IP_sum, how = 'outer') 
    FinalGroups_IP_sum = pd.DataFrame(Groups_IP_sum.groupby(['ip'])['numSum'].sum())
    FinalGroups_IP_sum.columns = [''.join(col) for col in FinalGroups_IP_sum.columns]
    FinalGroups_IP_sum = FinalGroups_IP_sum.reset_index()

    FinalGroups_IP_sum.sort_values(by=['numSum'],inplace=True,ascending=False)
    FinalGroups_IP_sum.reset_index(drop=True, inplace=True)

    ###Normalized to 20-30 for proper node size
    FinalGroups_IP_sum[['numSum']]=min_max_scaler_2030.fit_transform(FinalGroups_IP_sum[['numSum']])
    
    time_end1 = time.time() 
    cost1 = round((time_end1 - time_start1),4)
    
    #########Save the FinalGroups_IP_sum under different tau
    Path_FinalIP = "Results/FinalIPnNum_t%d.csv"%(t)
    FinalGroups_IP_sum.to_csv(Path_FinalIP,index=0)
    
########################For ROC&PRC
    #######Define Lists to keep TPR FPR R P
    TPRlist = []
    FPRlist = []
    Plist = []
    Rlist = []
    ########Draw ROC and PRC curves according to the selection of num, That is, the number of considered abnormal IPs
    num = 1 
    #At this point R TPR FPR are 0, and P is not defined  
    #So we define num to start from 1
    while num < FinalGroups_IP_sum.shape[0]+1:###+50 Ensure that all records can be obtained
        ab_finalsort= FinalGroups_IP_sum[0:num]
        num += 1
        dfabip = ab_finalsort['ip']
        if len(dfabip) == 0:
            continue
        dfabip.tolist()
        dfabip_set = set(dfabip)##Predicted set of abnormal IPs
        bingo=(dfabip_set)&(dfabcorrect_set)
        abIPi=len(dfabip_set)####Number of predicted abnormal IPs
        TPi=len(bingo)
        FNi=len(dfabcorrect)-TPi
        R,TPR,FPR,P = Metric(IPnumi,abIPi,TPi,FNi)
        TPRlist.append(TPR)
        FPRlist.append(FPR)
        Plist.append(P)
        Rlist.append(R)
    ROC = pd.DataFrame(FPRlist, columns=['FPR'])
    ROC = pd.concat([ROC, pd.DataFrame(TPRlist,columns=['TPR'])],axis=1)
    PR = pd.DataFrame(Rlist, columns=['R'])
    PR = pd.concat([PR, pd.DataFrame(Plist,columns=['P'])],axis=1)

    ####Importance drop_duplicates
    ROC=ROC.drop_duplicates()
    PR=PR.drop_duplicates()
    ####Importance sort_values
    ROC.sort_values(by=['FPR','TPR'],inplace=True,ascending=[True,True])
    PR.sort_values(by=['R','P'],inplace=True,ascending=[True,False])#########Same R, P starting from the higher
    ROC.reset_index(drop=True, inplace=True)
    PR.reset_index(drop=True, inplace=True)

    #####Importance Add the last line For ROC
    ROC.loc[ROC.shape[0]] = [1,ROC.loc[ROC.shape[0]-1,'TPR']]
    #####Importance Add the first line For PR
    PR.loc[-1] = [0,PR.loc[0,'P']]
    PR.index = PR.index + 1
    PR.sort_index(inplace=True)

    #####AUC
    ROC_AUC = np.trapz(list(ROC['TPR']),list(ROC['FPR'])).round(4)
    PRC_AUC = np.trapz(list(PR['P']),list(PR['R'])).round(4)
    
    ######Save Logs
    Path_ROC = "Results/ROC_t%d.csv"%(t)
    ROC.to_csv(Path_ROC,index=0)
    Path_PRC = "Results/PRC_t%d.csv"%(t)
    PR.to_csv(Path_PRC,index=0)
    

    
#########For Visualization#########Attack Graph
    time_start2 = time.time()
    
    FinalGroups_IP_sum['color']='#686789'
    for i in range(FinalGroups_IP_sum.shape[0]):
        if not (FinalGroups_IP_sum.iloc[i,0].startswith("10.")):
            FinalGroups_IP_sum.iloc[i,2]='#D89C7A'
    ##########node
    nodeslist = [{} for i in range(FinalGroups_IP_sum.shape[0])]
    for i in range(FinalGroups_IP_sum.shape[0]):
        nodeslist[i].update({'name':FinalGroups_IP_sum.iloc[i,0],'symbolSize':round(FinalGroups_IP_sum.iloc[i,1],2),
                             'itemStyle': {'color': FinalGroups_IP_sum.iloc[i,2]},
                             'label':{'normal':{"show":True,'fontSize':15,'color':"black"}},
                             "value":round(FinalGroups_IP_sum.iloc[i,1],2),
                            })
    ##########for links
    siplist=[[] for i in range(FinalGroups_IP_sum.shape[0])]
    diplist=[[] for i in range(FinalGroups_IP_sum.shape[0])]
    linknumlist=[[] for i in range(FinalGroups_IP_sum.shape[0])]
    timelist=[[] for i in range(FinalGroups_IP_sum.shape[0])]
    colorlist=[[] for i in range(FinalGroups_IP_sum.shape[0])]
    dictlist=[{}for i in range(FinalGroups_IP_sum.shape[0])]
    for i in range(FinalGroups_IP_sum.shape[0]):
        siplist[i].append(FinalGroups_IP_sum.iloc[i,0])
        for j in range(df_Keep_FinalAll_Front.shape[0]):
            if FinalGroups_IP_sum.iloc[i,0]==df_Keep_FinalAll_Front.iloc[j,1]:
                if df_Keep_FinalAll_Front.iloc[j,2] in diplist[i]:
                    pos = diplist[i].index(df_Keep_FinalAll_Front.iloc[j,2])
                    linknumlist[i][pos] += round(df_Keep_FinalAll_Front.iloc[j,3],2)
                    if linknumlist[i][pos] > 10:
                        linknumlist[i][pos] = 10####### Avoid too large connecting lines
                    timelist[i][pos] = timelist[i][pos]+","+df_Keep_FinalAll_Front.iloc[j,0]
                else:
                    diplist[i].append(df_Keep_FinalAll_Front.iloc[j,2])
                    linknumlist[i].append(round(df_Keep_FinalAll_Front.iloc[j,3],2))
                    timelist[i].append(df_Keep_FinalAll_Front.iloc[j,0])
                    colorlist[i].append("#686789")
        for j in range(df_Keep_FinalAll_Back.shape[0]):
            if FinalGroups_IP_sum.iloc[i,0]==df_Keep_FinalAll_Back.iloc[j,1]:
                if df_Keep_FinalAll_Back.iloc[j,2] in diplist[i]:
                    pos = diplist[i].index(df_Keep_FinalAll_Back.iloc[j,2])
                    linknumlist[i][pos] += round(df_Keep_FinalAll_Back.iloc[j,3],2)
                    timelist[i][pos] = timelist[i][pos]+","+df_Keep_FinalAll_Back.iloc[j,0]
                else:
                    diplist[i].append(df_Keep_FinalAll_Back.iloc[j,2])
                    linknumlist[i].append(round(df_Keep_FinalAll_Back.iloc[j,3],2))
                    timelist[i].append(df_Keep_FinalAll_Back.iloc[j,0])
                    colorlist[i].append("#AB545A")

        dictlist[i].update({'sip':siplist[i],'dip':diplist[i],"num":linknumlist[i],'time':timelist[i],'color':colorlist[i]})
    
    #######Visualization
    nodes = nodeslist
    links = []
    for i in range(len(dictlist)):
        sip = dictlist[i].get("sip")
        diplist = dictlist[i].get("dip")
        timelist = dictlist[i].get("time")
        numlist = dictlist[i].get("num")
        colorlist = dictlist[i].get("color")
        for j in range(len(diplist)):
            links.append({"source": sip, "target":diplist[j],"value":timelist[j],
                          'lineStyle': {'normal': {'curveness': 0,'color':colorlist[j],'width': numlist[j]}}
                         })
    g = (
        Graph()
        .add(
            "", 
            nodes, 
            links, 
            edge_symbol=['', 'arrow'],
            repulsion=900,
            gravity = 0.1,
            is_draggable=True,
            edge_length=270,
            edge_label=opts.LabelOpts(is_show=True, position="middle", formatter="{c}"),
        )

    )
    Path_html = "html/Graph_t%d.html"%(t)
    g.render(Path_html)
    
    time_end2 = time.time() 
    cost2 = round((time_end2 - time_start2),4)
#     g.render_notebook()

    Logdf = Logdf.append({'t':t,'ROC_AUC':ROC_AUC,'PRC_AUC':PRC_AUC,'AbIPFind_time':cost1,'Visual_time':cost2}, ignore_index=True)


d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

d:\anaconda\anaconda\lib\

In [9]:
Logdf

,t,ROC_AUC,PRC_AUC,AbIPFind_time,Visual_time
0,0.0,0.8571,0.8571,0.2920,0.0040
1,1.0,0.8571,0.8571,0.2830,0.0030
2,2.0,0.8571,0.8571,0.2803,0.0030
3,3.0,1.0000,1.0000,0.2850,0.0040
4,4.0,1.0000,1.0000,0.2950,0.0050
5,5.0,1.0000,1.0000,0.2946,0.0050
6,6.0,1.0000,1.0000,0.2881,0.0050
7,7.0,1.0000,1.0000,0.2880,0.0050
8,8.0,1.0000,1.0000,0.2930,0.0050
9,9.0,1.0000,1.0000,0.2994,0.0060


In [27]:
for i in [0,10,20,30]:
    print(i,"-",i+9,round(Logdf[i:i+10]['AbIPFind_time'].mean(),4))
print("40-50",round(Logdf[40:51]['AbIPFind_time'].mean(),4))

for i in [0,10,20,30]:
    print(i,"-",i+9,round(Logdf[i:i+10]['Visual_time'].mean(),4))
print("40-50",round(Logdf[40:51]['Visual_time'].mean(),4))

0 - 9 0.2898
10 - 19 0.2916
20 - 29 0.2812
30 - 39 0.2801
40-50 0.2814
0 - 9 0.0045
10 - 19 0.0064
20 - 29 0.0086
30 - 39 0.0113
40-50 0.0134


In [52]:
Logdf.to_csv("Results/All_Logs.csv",index=0)

### After Vis Check, you can exclude the other misidentified hosts

In [35]:
FinalGroups_IP_sum_t0 = pd.read_csv("Results/FinalIPnNum_t0.csv")

,ip,numSum
0,10.49.223.194,30.000
1,10.49.253.35,26.324
2,10.49.212.162,23.162
3,10.49.21.15,20.514
4,10.49.141.206,20.000
5,10.49.253.233,20.000


In [67]:
for i in range(26,Logdf_AfterVis.shape[0]):
    Logdf_AfterVis.iloc[i,1] = 1
    Logdf_AfterVis.iloc[i,2] = 1

Logdf_AfterVis.to_csv("Results/All_Logs_AfterVis.csv",index=0)
Logdf_AfterVis

,t,ROC_AUC,PRC_AUC
0,0.0,0.8571,0.8571
1,1.0,0.8571,0.8571
2,2.0,0.8571,0.8571
3,3.0,1.0000,1.0000
4,4.0,1.0000,1.0000
5,5.0,1.0000,1.0000
6,6.0,1.0000,1.0000
7,7.0,1.0000,1.0000
8,8.0,1.0000,1.0000
9,9.0,1.0000,1.0000
